In [173]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [174]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(
    Path('Resources/applicants_data.csv')
)

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [175]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [176]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN', 'NAME'])

# Review the DataFrame
applicant_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [177]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == 'object'].index)

# Display the categorical variables list
print(categorical_variables)

['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


In [178]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

In [179]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [180]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df 

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

In [181]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame 
num_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes != 'object'].index)
encoded_df = pd.concat([applicant_data_df[num_variables],encoded_df], axis=1)
# Review the Dataframe
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [182]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y.head()


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [183]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 6: Split the features and target sets into training and testing datasets.


In [184]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [185]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> We can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [186]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [187]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [188]:
# Define the number of hidden nodes for the first hidden layer
# Use the Python floor devision (//)to return the quotent
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2 

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

58

In [189]:
# Define the number of hidden nodes for the second hidden layer
# Use the Python floor devision (//)to return the quotent
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

29

In [190]:
# Create the Sequential model instance
neuron = Sequential()

In [191]:
# Add the first hidden layer
neuron.add(Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features,
             activation='relu')
      )

In [192]:
# Add the second hidden layer
neuron.add(Dense(units=hidden_nodes_layer2,
             activation='relu')
      )


In [193]:
# Add the output layer to the model specifying the number of output neurons and activation function
neuron.add(Dense(units=number_output_neurons,
             activation='sigmoid')
      )

In [194]:
# Display the Sequential model summary
neuron.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 58)                6786      
                                                                 
 dense_59 (Dense)            (None, 29)                1711      
                                                                 
 dense_60 (Dense)            (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [195]:
# Compile the Sequential model
neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [196]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50 )


Epoch 1/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5283 - accuracy: 0.7431
Epoch 2/50
804/804 [==============================] - 4s 6ms/step - loss: 0.5277 - accuracy: 0.7431
Epoch 3/50
804/804 [==============================] - 4s 6ms/step - loss: 0.5277 - accuracy: 0.7433
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5283 - accuracy: 0.7434
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5278 - accuracy: 0.7429
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5279 - accuracy: 0.7435
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5277 - accuracy: 0.7426
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5279 - accuracy: 0.7436
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5273 - accuracy: 0.7433
Epoch 10/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5271 - accuracy: 0.7439

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [197]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neuron.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7631 - accuracy: 0.4776 - 853ms/epoch - 3ms/step
Loss: 0.7630645632743835, Accuracy: 0.47755101323127747


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [41]:
# Set the model's file path
file_path = Path('Alternative_models')

# Export your model to a HDF5 file
neuron.save(file_path)


INFO:tensorflow:Assets written to: Alternative_models/assets


---

## Optimize the neural network model


### Step 1: Define three new deep neural network models. With each, try to improve on your first model’s predictive accuracy.

> Perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize the model for a predictive accuracy as close to 1 as possible, we can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1

In [198]:
# Define the the number of inputs (features) to the model
number_input_features_alt1 = len(X_train.iloc[0])

# Review the number of features
number_input_features_alt1

116

In [199]:
# Define the number of neurons in the output layer
number_output_neurons_alt1 = 1 

In [206]:
# Define the number of hidden nodes for all hidden layers
# Use the Python floor devision (//)to return the quotent
hidden_nodes_layer1_alt1 = (number_input_features_alt + number_output_neurons_alt1) // 2 
hidden_nodes_layer2_alt1 =  (hidden_nodes_layer1_alt1 + number_output_neurons_alt1) // 2
hidden_nodes_layer3_alt1 =  (hidden_nodes_layer2_alt1 + number_output_neurons_alt1) // 2

# Review the number of hidden nodes in all hidden layers
display(hidden_nodes_layer1_alt1)
display(hidden_nodes_layer2_alt1)
display(hidden_nodes_layer3_alt1)

58

29

15

In [207]:
# Create the Sequential model instance
nn_alt1 = Sequential()

In [208]:
# Add hidden layers
nn_alt1.add(Dense(units=hidden_nodes_layer1_alt1, input_dim=number_input_features, activation='relu'))
nn_alt1.add(Dense(units=hidden_nodes_layer2_alt1, activation='relu'))
nn_alt1.add(Dense(units=hidden_nodes_layer3_alt1, activation='relu'))

# Add output layer
nn_alt1.add(Dense(units=number_output_neurons_alt1, activation='sigmoid'))

In [209]:
# Check the structure of the model
nn_alt1.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 58)                6786      
                                                                 
 dense_71 (Dense)            (None, 29)                1711      
                                                                 
 dense_72 (Dense)            (None, 15)                450       
                                                                 
 dense_73 (Dense)            (None, 1)                 16        
                                                                 
Total params: 8,963
Trainable params: 8,963
Non-trainable params: 0
_________________________________________________________________


In [210]:
# Compile the Sequential model
nn_alt1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [211]:
# Fit the model using 50 epochs and the training data
fit_model_alt1 = nn_alt1.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5752 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5530 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5490 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5469 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5454 - accuracy: 0.7335
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5447 - accuracy: 0.7337
Epoch 7/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5442 - accuracy: 0.7353
Epoch 8/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5430 - accuracy: 0.7355
Epoch 9/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5425 - accuracy: 0.7364
Epoch 10/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5418 - accura

#### Alternative Model 2

In [216]:
# Define the the number of inputs (features) to the model
number_input_features_alt2 = len(X_train.iloc[0])

# Review the number of features
number_input_features_alt2

116

In [218]:
# Define the number of neurons in the output layer
number_output_neurons_alt2 = 1


In [221]:
# Define the number of hidden nodes for the first hidden layer
# Use the Python floor devision (//)to return the quotent
hidden_nodes_layer1_alt2 = (number_input_features_alt2 + number_output_neurons_alt2) // 2 
hidden_nodes_layer2_alt2 =  (hidden_nodes_layer1_alt2 + number_output_neurons_alt2) // 2
hidden_nodes_layer3_alt2 =  (hidden_nodes_layer2_alt2 + number_output_neurons_alt2) // 2

# Review the number of hidden nodes in all hidden layers
display(hidden_nodes_layer1_alt2)
display(hidden_nodes_layer2_alt2)
display(hidden_nodes_layer3_alt2)

58

29

15

In [222]:
# Create the Sequential model instance
nn_alt2 = Sequential()

In [223]:
# First hidden layer
nn_alt2.add(Dense(units=hidden_nodes_layer1_alt2, input_dim=number_input_features, activation='tanh'))
nn_alt2.add(Dense(units=hidden_nodes_layer2_alt2, activation='tanh'))
nn_alt2.add(Dense(units=hidden_nodes_layer3_alt2, activation='tanh'))

# Output layer
nn_alt2.add(Dense(units=number_output_neurons_alt2, activation='sigmoid'))

# Check the structure of the model
nn_alt2.summary()


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 58)                6786      
                                                                 
 dense_75 (Dense)            (None, 29)                1711      
                                                                 
 dense_76 (Dense)            (None, 15)                450       
                                                                 
 dense_77 (Dense)            (None, 1)                 16        
                                                                 
Total params: 8,963
Trainable params: 8,963
Non-trainable params: 0
_________________________________________________________________


In [224]:
# Compile the model
nn_alt2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [225]:
# Fit the model
fit_model_alt2 = nn_alt2.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 5s 5ms/step - loss: 0.5693 - accuracy: 0.7208
Epoch 2/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5526 - accuracy: 0.7297
Epoch 3/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5488 - accuracy: 0.7306
Epoch 4/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5465 - accuracy: 0.7329
Epoch 5/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5451 - accuracy: 0.7332
Epoch 6/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5430 - accuracy: 0.7342
Epoch 7/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5426 - accuracy: 0.7342
Epoch 8/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5412 - accuracy: 0.7353
Epoch 9/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5405 - accuracy: 0.7349
Epoch 10/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5398 - accura

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [226]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 1s - loss: 0.5687 - accuracy: 0.7294 - 765ms/epoch - 3ms/step
Loss: 0.5686784982681274, Accuracy: 0.7294460535049438


In [227]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_alt1.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 1s - loss: 0.5591 - accuracy: 0.7303 - 792ms/epoch - 3ms/step
Loss: 0.5590949058532715, Accuracy: 0.7302623987197876


In [228]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy =  nn_alt2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 1s - loss: 0.5593 - accuracy: 0.7303 - 874ms/epoch - 3ms/step
Loss: 0.559339702129364, Accuracy: 0.7302623987197876


### Step 3: Save each of your alternative models as an HDF5 file.


In [229]:
# Set the file path for the first alternative model
file_path = Path("alternative_models/Model_1.h5")

# Export your model to a HDF5 file
nn_alt1.save(file_path)

In [230]:
# Set the file path for the second alternative model
file_path = Path("alternative_models/Model_2.h5")

# Export your model to a HDF5 file
nn_alt2.save(file_path)